# Library Import

In [219]:
import os
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np

In [220]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, GlobalMaxPooling1D
from keras.layers.convolutional import Conv1D, MaxPooling1D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from keras.utils import np_utils

# Data Load

In [221]:
path_fault2_ex = 'C:/Users/baek/Desktop/NURO/data_cycle/abnormal2/external'
path_fault2_in = 'C:/Users/baek/Desktop/NURO/data_cycle/abnormal2/internal'

path_fault1_ex = 'C:/Users/baek/Desktop/NURO/data_cycle/abnormal1/external'
path_fault1_in = 'C:/Users/baek/Desktop/NURO/data_cycle/abnormal1/internal'

path_normal_ex = 'C:/Users/baek/Desktop/NURO/data_cycle/normal/external'
path_normal_in = 'C:/Users/baek/Desktop/NURO/data_cycle/normal/internal'

In [264]:
dat_f1 = [[] for i in range(0,2)]
dat_f2 = [[] for i in range(0,2)]
dat_n = []

In [265]:
for i in range(len(os.listdir(path_fault1_ex))):
    temp =pd.read_csv(path_fault1_ex + '/'+ os.listdir(path_fault1_ex)[i], sep = '\t' )
    dat_f1[0].append(temp)
    
    
for i in range(len(os.listdir(path_fault1_in))):
    temp = pd.read_csv(path_fault1_in + '/'+os.listdir(path_fault1_in)[i], sep = ',' , low_memory=False)
    dat_f1[1].append(temp)

    
for i in range(len(os.listdir(path_fault2_ex))):
    temp =pd.read_csv(path_fault2_ex + '/'+ os.listdir(path_fault2_ex)[i], sep = '\t' )
    dat_f2[0].append(temp)
    
    
for i in range(len(os.listdir(path_fault2_in))):
    temp = pd.read_csv(path_fault2_in + '/'+os.listdir(path_fault2_in)[i], sep = ',' , low_memory=False)
    dat_f2[1].append(temp)
    
    
for i in range(len(os.listdir(path_normal_ex))):
    temp =pd.read_csv(path_normal_ex + '/'+os.listdir(path_normal_ex)[i], sep = '\t' )
    dat_n.append(temp)

In [353]:
data_f1 = []
for i in range(len(dat_f1[0])):
    temp = dat_f1[0][i].loc[:,['ax(g)','ay(g)','az(g)']] - dat_f1[0][i].loc[:,['ax(g)','ay(g)','az(g)']].median()
    temp = temp.loc[np.unique(np.around(np.array([i for i in range(len( dat_f1[1][i].loc[dat_f1[1][i]['CoreSN'] == 'core0'+str(2),:]))]) / len( dat_f1[1][i].loc[dat_f1[1][i]['CoreSN'] == 'core0'+str(2),:]) * len(temp)).astype('int')).tolist()].reset_index().drop(labels = ['index'], axis=1)
    data_f1.append(temp)
    
data_f2 = []
for i in range(len(dat_f2[0])):
    temp = dat_f2[0][i].loc[:,['ax(g)','ay(g)','az(g)']] - dat_f2[0][i].loc[:,['ax(g)','ay(g)','az(g)']].median()
    temp = temp.loc[np.unique(np.around(np.array([i for i in range(len(dat_f2[1][i].loc[dat_f2[1][i]['CoreSN'] == 'core0'+str(2),:]))]) / len(dat_f2[1][i].loc[dat_f2[1][i]['CoreSN'] == 'core0'+str(2),:]) * len(temp)).astype('int')).tolist()].reset_index().drop(labels = ['index'], axis=1)
    data_f2.append(temp)
    
data_n = []
for i in range(len(dat_n)):
    temp = dat_n[i].loc[:,['ax(g)','ay(g)','az(g)']] - dat_n[i].loc[:,['ax(g)','ay(g)','az(g)']].median()
    temp = temp.loc[np.unique(np.around(np.array([i for i in range(len( dat_f1[1][i].loc[dat_f1[1][i]['CoreSN'] == 'core0'+str(1),:]))]) / len( dat_f1[1][i].loc[dat_f1[1][i]['CoreSN'] == 'core0'+str(1),:]) * len(temp)).astype('int')).tolist()].reset_index().drop(labels = ['index'], axis=1)
    data_n.append(temp)
    
##temp = temp.loc[np.unique(np.around(np.array([i for i in range(min_data_len)]) / min_data_len * len(temp)).astype('int')).tolist()].reset_index()

In [342]:
d_n = []
d_f1 = []
d_f2 = []



for i in range(0,5):
    scaler_ = MinMaxScaler()
    scaler_.fit(data_n[i])
    data_n[i] = scaler_.transform(data_n[i])
    for j in range(len(data_n[i])//300):
        d_n.append(data_n[i][300*j:300*(j+1),:])

for i in range(0,5):
    scaler_ = MinMaxScaler()
    scaler_.fit(data_f1[i])
    data_f1[i] = scaler_.transform(data_f1[i])
    for j in range(len(data_f1[i])//300):
        d_f1.append(data_f1[i][300*j:300*(j+1),:])
        
for i in range(0,5):
    scaler_ = MinMaxScaler()
    scaler_.fit(data_f2[i])
    data_f2[i] = scaler_.transform(data_f2[i])
    for j in range(len(data_f2[i])//300):
        d_f2.append(data_f2[i][300*j:300*(j+1),:])


In [366]:
label_n = [0 for i in range(len(d_n))]
label_f1 = [1 for i in range(len(d_f1))]
label_f2 = [2 for i in range(len(d_f2))]


In [367]:
d_n.extend(d_f1)
label_n.extend(label_f1)

d_n.extend(d_f2)
label_n.extend(label_f2)

In [368]:
X_train, X_test, y_train, y_test = train_test_split(d_n,label_n,test_size=0.2, shuffle=True, stratify=label_n, random_state=42)


In [370]:
for i in range(len(X_train)):
    X_train[i] = np.array(X_train[i])
    
for i in range(len(X_test)):
    X_test[i] = np.array(X_test[i])
    
X_test = np.array(X_test)
X_train = np.array(X_train)
y_test = np.array(y_test)
y_train = np.array(y_train)

X_test = X_test.reshape(len(X_test),300,3,1)
X_train = X_train.reshape(len(X_train),300,3,1)

y_train_hot = np_utils.to_categorical(y_train, 3)
y_test_hot= np_utils.to_categorical(y_test, 3)

# Train & Validataion

In [373]:
model =  Sequential()
model.add(Conv1D(filters = 32, kernel_size = 2, activation = 'relu', input_shape =(300,3)))
model.add(MaxPooling1D())

model.add(Conv1D(32, 3, padding='valid', activation='relu'))
model.add(MaxPooling1D())

model.add(Conv1D(64, 3, padding='valid', activation='relu'))
model.add(MaxPooling1D())

model.add(Conv1D(128, 3, padding='valid', activation='relu'))
model.add(MaxPooling1D())

model.add(Flatten())
model.add(Dense(2176, activation = 'relu'))
model.add(Dense(50, activation = 'relu'))
model.add(Dense(3, activation = 'softmax'))

model.compile(optimizer = 'adam', loss=  'categorical_crossentropy', metrics = ['acc'])

In [374]:
history =  model.fit(X_train, y_train_hot, epochs = 100,batch_size = 32, validation_data = ((X_test), y_test_hot),callbacks=[EarlyStopping(monitor = 'val_loss', min_delta = 0, patience = 30, mode = 'auto')])

Epoch 1/100
48/48 [==============================] - 3s 45ms/step - loss: 0.8930 - acc: 0.5674 - val_loss: 0.6137 - val_acc: 0.7995
Epoch 2/100
48/48 [==============================] - 2s 43ms/step - loss: 0.5536 - acc: 0.7948 - val_loss: 0.4649 - val_acc: 0.8333
Epoch 3/100
48/48 [==============================] - 2s 42ms/step - loss: 0.5212 - acc: 0.7928 - val_loss: 0.4088 - val_acc: 0.8359
Epoch 4/100
48/48 [==============================] - 2s 42ms/step - loss: 0.3667 - acc: 0.8534 - val_loss: 0.3954 - val_acc: 0.8307
Epoch 5/100
48/48 [==============================] - 2s 42ms/step - loss: 0.3488 - acc: 0.8632 - val_loss: 0.3607 - val_acc: 0.8516
Epoch 6/100
48/48 [==============================] - 2s 42ms/step - loss: 0.2749 - acc: 0.8912 - val_loss: 0.2533 - val_acc: 0.9089
Epoch 7/100
48/48 [==============================] - 2s 42ms/step - loss: 0.3318 - acc: 0.8834 - val_loss: 0.2471 - val_acc: 0.9010
Epoch 8/100
48/48 [==============================] - 2s 42ms/step - loss: 0.

# Test

In [392]:
d_n_test = []
d_f1_test = []
d_f2_test = []



for i in range(5,9):
    scaler_ = MinMaxScaler()
    scaler_.fit(data_n[i])
    data_n[i] = scaler_.transform(data_n[i])
    for j in range(len(data_n[i])//300):
        d_n_test.append(data_n[i][300*j:300*(j+1),:])

for i in range(5,9):
    scaler_ = MinMaxScaler()
    scaler_.fit(data_f1[i])
    data_f1[i] = scaler_.transform(data_f1[i])
    for j in range(len(data_f1[i])//300):
        d_f1_test.append(data_f1[i][300*j:300*(j+1),:])
        
for i in range(5,9):
    scaler_ = MinMaxScaler()
    scaler_.fit(data_f2[i])
    data_f2[i] = scaler_.transform(data_f2[i])
    for j in range(len(data_f2[i])//300):
        d_f2_test.append(data_f2[i][300*j:300*(j+1),:])


In [393]:
label_n_test = [0 for i in range(len(d_n_test))]
label_f1_test = [1 for i in range(len(d_f1_test))]
label_f2_test = [2 for i in range(len(d_f2_test))]


In [394]:
d_n_test.extend(d_f1_test)
label_n_test.extend(label_f1_test)

d_n_test.extend(d_f2_test)
label_n_test.extend(label_f2_test)

In [395]:
X_train, X_test, y_train, y_test = train_test_split(d_n_test,label_n_test,test_size=0.01, shuffle=True, stratify=label_n_test, random_state=42)


In [396]:
for i in range(len(X_train)):
    X_train[i] = np.array(X_train[i])
    
for i in range(len(X_test)):
    X_test[i] = np.array(X_test[i])
    
X_test = np.array(X_test)
X_train = np.array(X_train)
y_test = np.array(y_test)
y_train = np.array(y_train)

X_test = X_test.reshape(len(X_test),300,3,1)
X_train = X_train.reshape(len(X_train),300,3,1)


In [397]:
y_train_hot = np_utils.to_categorical(y_train, 3)
y_test_hot= np_utils.to_categorical(y_test, 3)

In [399]:
model.predict(X_train)

array([[3.8146069e-07, 9.9977380e-01, 2.2586010e-04],
       [2.9617557e-03, 9.9703825e-01, 3.4045194e-10],
       [1.5026442e-08, 1.0000000e+00, 1.0921709e-11],
       ...,
       [9.9999881e-01, 1.1485681e-06, 2.5665754e-11],
       [5.7302415e-08, 9.9999988e-01, 1.7228500e-07],
       [9.9863833e-01, 1.3616360e-03, 2.4367586e-10]], dtype=float32)

In [398]:
results = model.evaluate(X_train,  y_train_hot)
print('Test accuracy: ', results[1])

54/54 [==============================] - 1s 7ms/step - loss: 5.9684 - acc: 0.5125
Test accuracy:  0.5125364661216736
